In [ ]:
import streamlit as st
import os
from IPython.display import Markdown

# Librerías para la preparación de datos
from langchain.document_loaders import PyPDFDirectoryLoader #leer un pdf 
from langchain.text_splitter import RecursiveCharacterTextSplitter #para hacer chunks
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings #obtener embedings

# Librerías para el proceso de Retrieval
from langchain import hub #para traernos el prompt
from langchain_core.output_parsers import StrOutputParser #la respuesta que devuelve Gemini, no tengas que estar buscando el texto
from langchain_core.runnables import RunnablePassthrough 
from langchain_google_genai import ChatGoogleGenerativeAI #clase del LLM que estamos usando

st.set_page_config(
    page_title="LangChain APP",
    page_icon="📖",
    layout="wide",
    initial_sidebar_state="expanded",
)
#2. CONFIGURACION INICIAL DE CARPETAS
import os
os.mkdir("content")
os.mkdir("content/MisDatos")
os.mkdir("content/VectorDB")

from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"

#3. PREPARACIÓN DE LOS DATOS
source_data_folder = "./content/MisDatos"
# Leyendo los PDFs del directorio configurado
loader = PyPDFDirectoryLoader(source_data_folder)
data_on_pdf = loader.load()
# cantidad de data cargada
len(data_on_pdf)


# Particionando los datos. Con un tamaño delimitado (chunks) y 
# 200 caracters de overlapping para preservar el contexto
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""], #para que corte las frases por ahi
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(data_on_pdf)
# Cantidad de chunks obtenidos
len(splits)

#Generar los embeddings y almacenarlos en Chroma DB.

from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings

# Crea la instancia de embeddings con Cohere
embeddings_model = CohereEmbeddings(cohere_api_key=os.environ["API_KEY_COHERE"], user_agent="antonio")

path_db = "./content/VectorDB"  # Ruta a la base de datos del vector store

# Crear el vector store a partir de tus documentos 'splits'
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings_model, 
    persist_directory=path_db
)

#4. CONFIGURACIÓN DEL RETRIEVAL

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.environ["API_KEY_GOOGLE"])

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    # Funcion auxiliar para enviar el contexto al modelo como parte del prompt
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)